# Monte Carlo Project

## Iacob Jessica, Mourre Grégoire

In this Jupter file, we will implement the different methods presented and described in https://github.com/gregoiremrr/Monte-Carlo-for-American-Options.


In [106]:
import numpy as np
from numpy.random import default_rng
from scipy.optimize import curve_fit
from tqdm import tqdm

# Longstaff and Schwartz method for the lower bound

In [107]:
n = 10000
m = 12
rng = default_rng()

r, sigma, x, K, T = 0, .3, 100, 105, 1
dt = T/m

def g(x, K=K):
    return np.maximum(K-x,0)

def phi(x):
    return np.array([1, x, x**2, x**3, x**4, g(x), g(x)**2, g(x)**3, g(x)**4], dtype=object)
l = len(phi(0))

def reg(x, *alphas):
    alpha = np.array(alphas)
    return np.sum(alpha * phi(x))

In [108]:
# Tsitsiklis & Van Roy's algorithm

# Step 1
B = rng.normal(size=m*n).reshape(m,n)
X = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B]), axis=0)

# Step 2
V = np.zeros(n*(m+1)).reshape(m+1,n)
V[-1,:] = g(X[-1,:])

# Step 3
alpha0 = np.zeros(l)
for i in range(m-1, 0, -1):
    alpha_star = curve_fit(reg, X[i,:], V[i+1,:], p0=alpha0)[0]
    _1 = g(X[i,:])
    _2 = reg(X[i,:], alpha_star)
    V[i,:] = _1 * (_1 >= _2) + _2 * (_1 < _2)

# Step 4
_1 = g(x)
_2 = np.mean(V[1,:])
V0 = _1 * (_1 >= _2) + _2 * (_1 < _2)

print("Estimator:", V0)

Estimator: 14.770135278078465


In [109]:
# Longstaff & Schwartz's algorithm

# Step 2
V2 = np.zeros(n*(m+1)).reshape((m+1),n)
V2[-1,:] = g(X[-1,:])

alpha_tau = np.zeros(l*(m+1)).reshape((m+1),l)

# Step 3
alpha0 = np.zeros(l)
for i in range(m-1, 0, -1):
    alpha_tau[i,:] = curve_fit(reg, X[i,:], V2[i+1,:], p0=alpha0)[0]
    _1 = g(X[i,:])
    _2 = reg(X[i,:], alpha_tau[i,:])
    V2[i,:] = _1 * (_1 >= _2) + V2[(i+1),:] * (_1 < _2)

# Step 4 (not used)
_1 = g(x)
_2 = np.mean(V2[1,:])
V02 = np.mean(_1 * (_1 >= _2) + V2[1,:] * (_1 < _2))

# Step 5
B2 = rng.normal(size=m*n).reshape(m,n)
X2 = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B2]), axis=0)

V3 = np.zeros(n*(m+1)).reshape((m+1),n)
V3[-1,:] = g(X2[-1,:])

for i in range(m-1, 0, -1):
    _1 = g(X2[i,:])
    _2 = reg(X2[i,:], alpha_tau[i,:])
    V3[i,:] = _1 * (_1 >= _2) + V3[(i+1),:] * (_1 < _2)

_1 = g(x)
_2 = np.mean(V3[1,:])
V03_ = _1 * (_1 >= _2) + V3[1,:] * (_1 < _2)

V03 = np.mean(V03_)
std = np.std(V03_)
conv_interval = V03 + np.array([-1,1]) * 1.96 * std / np.sqrt(n)
final_lowerbound = conv_interval[0]

print("Estimator:", V03)
print("Standard deviation:", std / np.sqrt(n))
print("Condidence interval 95%:", conv_interval)
print("Error:", 100 * 1.96 * std / (V03 * np.sqrt(n)), "%")

Estimator: 15.045031312669142
Standard deviation: 0.15352993479219845
Condidence interval 95%: [14.74411264 15.34594998]
Error: 2.0001199461732657 %


# Martingales from Approximate Value Functions

In [110]:
# One trajectory

n2 = n

B_upperbound = rng.normal(size=m)
X_upperbound = np.cumprod(np.concatenate([[x], 1 + r * dt + sigma * np.sqrt(dt) * B_upperbound]), axis=0)
Normal = rng.normal(size=m*n2).reshape(m,n2)
M = np.zeros(m+1)
Mi = 0

for i in range(1, m):
    V_upperbound = max(g(X_upperbound[i]), reg(X_upperbound[i], alpha_tau[i,:]))
    X_next = X_upperbound[i-1] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:])
    V_Ynext = np.maximum(g(X_next), reg(X_next, alpha_tau[i,:]))
    delta = V_upperbound - np.mean(V_Ynext)
    Mi += delta
    M[i] = Mi

X_next = X_upperbound[-2] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:])
delta = g(X_upperbound[-1]) - np.mean(g(X_next))
Mi += delta
M[-1] = Mi

V0_upperbound = np.max(g(X_upperbound) - M)
print(V0_upperbound)

14.66132934328855


In [111]:
# Monte-Carlo method for n trajectories

n2 = 1000

Normal = rng.normal(size=m*n*n2).reshape(m,n2,n)
M = np.zeros(n*(m+1)).reshape(m+1,n)
Mi = np.zeros(n)

for i in tqdm(range(1, m)):
    V_upperbound = np.maximum(g(X2[i,:]), reg(X2[i,:], alpha_tau[i,:]))
    X_next = X2[i-1,:] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:,:])
    V_Ynext = np.maximum(g(X_next), reg(X_next, alpha_tau[i,:]))
    Mi += V_upperbound - np.mean(V_Ynext, axis=0)
    M[i,:] = Mi

X_next = X2[-2,:] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:,:])
Mi += g(X2[-1,:]) - np.mean(g(X_next), axis=0)
M[-1,:] = Mi

V0_upperbound_ = np.max(g(X2) - M, axis=0)
V0_upperbound = np.mean(V0_upperbound_)
std_upperbound = np.std(V0_upperbound_)
conv_interval_upperbound = V0_upperbound + np.array([-1,1]) * 1.96 * std_upperbound / np.sqrt(n)
final_upperbound = conv_interval_upperbound[1]

print("Estimator:", V0_upperbound)
print("Standard deviation:", std_upperbound / np.sqrt(n))
print("Condidence interval 95%:", conv_interval_upperbound)
print("Error:", 100 * 1.96 * std_upperbound / (V0_upperbound * np.sqrt(n)), "%")

100%|██████████| 11/11 [00:06<00:00,  1.80it/s]

Estimator: 14.998667126467112
Standard deviation: 0.007664949983179101
Condidence interval 95%: [14.98364382 15.01369043]
Error: 0.10016424686511281 %


In [112]:
print("Confidence interval (lower & upper bounds):", [final_lowerbound, final_upperbound])

Confidence interval (lower & upper bounds): [14.744112640476432, 15.013690428434142]


# Martingales from Stopping Rules

In [113]:
# Step 1
n_subpaths = 500

Normal = rng.normal(size=m*m*n*n_subpaths).reshape(n,m,m,n_subpaths)

subpaths = np.zeros(m*(m+1)*n*n_subpaths).reshape(n,m,m+1,n_subpaths)
for k in tqdm(range(n)):
    for i in range(m):
        subpaths[k,i,:i+1,:] = np.repeat(X2[:i+1,k], n_subpaths).reshape(i+1,-1)
        subpaths[k,i,i+1:,:] = X2[i,k] * np.cumprod(1 + r * dt + sigma * np.sqrt(dt) * Normal[k,i,i:,:], axis=0)

subpaths_values = np.zeros(m*(m+1)*n*n_subpaths).reshape(n,m,m+1,n_subpaths)
for k in tqdm(range(n)):
    for i in range(m):
        subpaths_values[k,i,m,:] = g(subpaths[k,i,m,:])
        for j in range(m-1,i,-1):
            _1 = g(subpaths[k,i,j,:])
            _2 = reg(subpaths[k,i,j,:], alpha_tau[j,:])
            subpaths_values[k,i,j,:] =  _1 * (_1 >= _2) + subpaths_values[k,i,j+1,:] * (_1 < _2)

# approximation of E[ h_{\tau_{i+1}}(X_{\tau_{i+1}}) | X_i ] = V_iplus1[:,i]
V_iplus1 = np.zeros(n*m).reshape(n,m)
for i in range(m):
    V_iplus1[:,i] = np.mean(subpaths_values[:,i,i+1,:], axis = 1)

# approximation of E[ h_{\tau_i}(X_{\tau_i}) | X_i ] = V_i[:,i]
V_i = np.zeros(n*(m+1)).reshape(n,m+1)
for i in range(1,m):
    _1 = g(X2[i,:])
    _2 = reg(X2[i,:], alpha_tau[i,:])
    V_i[:,i] = _1 * (_1 >= _2) + V_iplus1[:,i] * (_1 < _2)

V_i[:,m] = g(X2[m,:])

Mk = np.zeros(n)
M2 = np.zeros(n*(m+1)).reshape(n,m+1)
for i in range(1,m+1):
    Mk += V_i[:,i] - V_iplus1[:,i-1]
    M2[:,i] = Mk

V0_upperbound2_ = np.max(g(X2) - M2.T, axis=0)
V0_upperbound2 = np.mean(V0_upperbound2_)
std_upperbound2 = np.std(V0_upperbound2_)
conv_interval_upperbound2 = V0_upperbound2 + np.array([-1,1]) * 1.96 * std_upperbound2 / np.sqrt(n)
final_upperbound2 = conv_interval_upperbound2[1]

print("Estimator:", V0_upperbound2)
print("Standard deviation:", std_upperbound2 / np.sqrt(n))
print("Condidence interval 95%:", conv_interval_upperbound2)
print("Error:", 100 * 1.96 * std_upperbound2 / (V0_upperbound2 * np.sqrt(n)), "%")

100%|██████████| 10000/10000 [00:39<00:00, 251.96it/s]


Estimator: 15.04229593718883
Standard deviation: 0.007309245963377969
Condidence interval 95%: [15.02796982 15.05662206]
Error: 0.09523893259407677 %


In [114]:
print("Confidence interval (lower & upper bounds):", [final_lowerbound, final_upperbound2])

Confidence interval (lower & upper bounds): [14.744112640476432, 15.056622059277052]


# Finite difference